In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%%capture
! pip install datasets

## 1. Semantic Search

In [3]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')
sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

README.md:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

...

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
klue_mrc_dataset = klue_mrc_dataset.train_test_split(train_size=1000, shuffle=False)['train']

embeddings = sentence_model.encode(klue_mrc_dataset['context'])
embeddings.shape

(1000, 768)

In [5]:
%%capture
! pip install faiss-cpu

In [6]:
import faiss

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [7]:
query = "이번 연도에는 언제 비가 많이 올까?"
query_embedding = sentence_model.encode([query])
distances, indices = index.search(query_embedding, 3)

print(f'{query}\n')
for idx in indices[0]:
  print(klue_mrc_dataset['context'][idx][:50])

이번 연도에는 언제 비가 많이 올까?

올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 
연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그
연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그


In [8]:
query = klue_mrc_dataset[3]['question']
query_embedding = sentence_model.encode([query])
distances, indices = index.search(query_embedding, 3)

print(f'{query}\n')
for idx in indices[0]:
  print(klue_mrc_dataset['context'][idx][:50])

로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?

태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 
태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 
미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스


## 2. Sentence-Transformers in LlamaIndex

In [9]:
%%capture
! pip install llama_index llama-index-embeddings-huggingface

In [10]:
from llama_index.core import VectorStoreIndex, Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.settings import Settings

embed_model = HuggingFaceEmbedding(model_name='snunlp/KR-SBERT-V40K-klueNLI-augSTS')
Settings.embed_model = embed_model

text_list = klue_mrc_dataset[:100]['context']
documents = [Document(text=t) for t in text_list]

index_llama = VectorStoreIndex.from_documents(documents)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
query = "이번 연도에는 언제 비가 많이 올까?"
query_embedding = embed_model.get_text_embedding(query)

retriever = index_llama.as_retriever(top_k=3)
retrieved_documents = retriever.retrieve(query)

print(f'{query}\n')
for doc in retrieved_documents[:3]:
    print(doc.text[:50])

이번 연도에는 언제 비가 많이 올까?

올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 
토즈 스터디센터가 ‘겨울, 실력을 만들다’ 캠페인을 진행한다. 이 캠페인은 느슨해지거나 학


In [12]:
query = klue_mrc_dataset[3]['question']
query_embedding = embed_model.get_text_embedding(query)

retriever = index_llama.as_retriever(top_k=3)
retrieved_documents = retriever.retrieve(query)

print(f'{query}\n')
for doc in retrieved_documents[:3]:
    print(doc.text[:50])

로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?

미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스
태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 
